# Dictionary-Based Time Series Classification

Dictionary-based methods discretize subsequences into symbolic words, then classify using a
bag-of-words representation. Common variants include BOSS and WEASEL-style pipelines.

## Core idea
Slide a window of length $w$ across the series, discretize each window into a word, and count
word frequencies. The feature vector is the word histogram
$\mathbf{h}(x) = (c_1, c_2, \dots, c_V)$, which can be fed to a linear or ensemble classifier.

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
from collections import Counter

np.random.seed(23)
t = np.linspace(0, 2 * np.pi, 80)
x = np.sin(t) + 0.1 * np.random.randn(len(t))

alphabet = np.array(list('abcd'))

def to_symbols(window, n_bins=4):
    z = (window - window.mean()) / (window.std(ddof=0) + 1e-9)
    bins = np.quantile(z, np.linspace(0, 1, n_bins + 1)[1:-1])
    idx = np.digitize(z, bins)
    return ''.join(alphabet[idx])

w = 12
words = []
for start in range(0, len(x) - w + 1):
    words.append(to_symbols(x[start : start + w]))

counts = Counter(words)
hist_df = pd.DataFrame({'word': list(counts.keys()), 'count': list(counts.values())})
hist_df = hist_df.sort_values('count', ascending=False).head(12)

fig = px.bar(hist_df, x='word', y='count', title='Top symbolic words from sliding windows')
fig

## sktime inventory for dictionary-based classifiers
Dictionary-based estimators typically live in modules containing `dictionary_based`.

In [ ]:
try:
    import pandas as pd
    from sktime.registry import all_estimators

    def _estimators_df():
        try:
            return all_estimators(estimator_types='classifier', as_dataframe=True)
        except TypeError:
            ests = all_estimators(estimator_types='classifier')
            return pd.DataFrame(
                [{'name': n, 'class': c.__name__, 'module': c.__module__} for n, c in ests]
            )

    df = _estimators_df()
    mask = df['module'].str.contains('dictionary', case=False, na=False)
    df_dict = df[mask].sort_values('name')
    df_dict
except Exception as exc:
    print('sktime not installed or registry unavailable:', exc)


## When to use
- Works well when local motifs repeat and can be discretized into symbolic words.
- Histogram features are sparse and fast for linear classifiers.
- Sensitive to window length and discretization choices; tune carefully.